In [ ]:
!pip install -q datasets #==1.9.0
!pip install -q transformers #==4.10.3
import transformers
import torch
import pickle
import pandas as pd
import numpy as np
from numpy import savetxt, loadtxt
import os
import json
from datasets import load_dataset
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.0 MB/s eta 0:00:00


# Set configuration

In [ ]:
def Fine_tune_func():
  ftn = int(input(f"please enter 1 if you are in the fine-tune mode, otherwise enter 0: "))
  while ftn not in [0,1]:
    print(f"invalid input, try again!")
    ftn = int(input(f"please enter 1 if you are in the fine-tune mode, otherwise enter 0: "))
  return ftn

In [ ]:
def check_for_Modes():
  '''
  This function is explicitly designed for single-lingual mode, and will be used in the time of needs.
  '''
  # Receiving the Mode (Single/Multi lingual)
  Mode = int(input(f"Enter 1 if you are surfing on the single-lingual mode, otherwise consider this as irrelevent by entering 0: "))
  while Mode not in [0,1]:
    print(f"invalid input, try again!")
    Mode = int(input(f"Enter 1 if you are surfing on the single-lingual mode, otherwise consider this as irrelevent by entering 0: "))
  if Mode:
    # Receiving the code per lingual.
    code = int(input(f"please enter 0 for (French2), enter 1 for (English2), and 2 for (Farsi): "))
    while code not in [0,1,2]:
      print(f"invalid input, try again!")
      code = int(input(f"please enter 0 for (French2), enter 1 for (English2), and 2 for (Farsi): "))
    return code
  else:
    return "Multi-lingual"

In [ ]:
def Combination_def():
  comb = int(input(f"please enter 0 for (French2 & English2), 1 for (English2 & Farsi), and 2 for (French2 & Farsi): "))
  while comb not in [0,1,2]:
    print(f"invalid input, try again!")
    comb = int(input(f"please enter 0 for (French2 & English2), 1 for (English2 & Farsi), and 2 for (French2 & Farsi): "))

  if comb == 0:
    dataset_code = [0,1] # French2 & English2
  elif comb == 1:
    dataset_code = [1,2] # English2 & Farsi
  else:
    dataset_code = [0,2] # French2 & Farsi

  return comb, dataset_code

In [ ]:
def set_path(Fine_tune, Combination, dataset_code, from_gdrive=True):

  if (from_gdrive):
    Actual_path = "/content/drive/Shareddrives/Gdrive/NLP Bachelors' Project/checkpoint/"
  else:
    Actual_path = "/content/drive/MyDrive/NLP Bachelors' Project/checkpoint/"


  if not isinstance(dataset_code, int):

    Normal_path = Actual_path + "MLL"
    Fine_tune_path = Actual_path + "Fine-tune/MLL"

    if Fine_tune:
        if Combination == 0:

            path_to_checkpoint = Fine_tune_path + "(English2,French2)"
            path_to_pretrained_weight = Normal_path + "(English2,French2)"

        elif Combination == 1:

            path_to_checkpoint = Fine_tune_path + "(Farsi,English2)"
            path_to_pretrained_weight = Normal_path + "(Farsi,English2)"

        elif Combination == 2:

            path_to_checkpoint = Fine_tune_path + "(French2,Farsi)"
            path_to_pretrained_weight = Normal_path + "(French2,Farsi)"

    else:
        path_to_pretrained_weight = None
        if Combination == 0:

            path_to_checkpoint = Normal_path + "(English2,French2)"

        elif Combination == 1:

            path_to_checkpoint = Normal_path + "(Farsi,English2)"

        elif Combination == 2:

            path_to_checkpoint = Normal_path + "(French2,Farsi)"
  else:
    Normal_path = Actual_path + "SLL"
    path_to_pretrained_weight = None
    if dataset_code == 0:
      path_to_checkpoint = Normal_path + "(French2)"
    elif dataset_code == 1:
      path_to_checkpoint = Normal_path + "(English2)"
    else:
      path_to_checkpoint = Normal_path + "(Farsi)"

  return path_to_checkpoint, path_to_pretrained_weight

In [ ]:
def set_status():
  Fine_tune = Fine_tune_func()
  #Combination = None
  Combination, dataset_code = Combination_def() # dataset_code in multi_lingual mode is list of two items, whereas in single-lingual mode we have a single integer.

  return Fine_tune, Combination, dataset_code


In [ ]:
def return_last_checkpoint(path):
  if (path and len(os.listdir(path)) > 0):
    lst = os.listdir(path)
    lst = [item for item in lst if item[:10]=='checkpoint']
    num = [int(item[11:]) for item in lst]
    return lst[num.index(max(num))]
  else:
    return None

In [ ]:
def return_model_name(Fine_tune, path, Last_checkpoint):
    model_name = "xlm-roberta-base"
    return model_name
  # if Fine_tune == 0:
  #   model_name = "bert-base-uncased"
  # else:
  #   model_name = path + "/" + Last_checkpoint
  #   json_filename = model_name + "/config.json"
  #   with open(json_filename) as json_file:
  #     json_decoded = json.load(json_file)

  #   json_decoded['model_type'] = 'bert'

  #   with open(json_filename, 'w') as json_file:
  #     json.dump(json_decoded, json_file, indent=2, separators=(',', ': '))

  # return model_name

# Dataset setup

In [ ]:
def load_dict():
    import pickle
    # dataset = load_dataset("AmazonScience/massive", name='en-US')
    dataset = load_dataset("AmazonScience/massive", name='fa-IR')
    # dataset = load_dataset("AmazonScience/massive", name='fr-FR')
    # with open(f"/content/drive/MyDrive/NLP Bachelors' Project/en2_dataset.pkl", 'rb') as f:
        # massive_en = pickle.load(f)
    with open(f"/content/drive/MyDrive/NLP Bachelors' Project/fa_dataset.pkl", 'rb') as f:
        massive_fa = pickle.load(f)
    # with open(f"/content/drive/MyDrive/NLP Bachelors' Project/fr_dataset.pkl", 'rb') as f:
        # massive_fr = pickle.load(f)

    dataset = load_dataset("clinc_oos", name="small")
    dataset = dataset.shuffle(seed=42)
    massive_en = dataset.rename_column("text", "utt")

    dataset = load_dataset("miam", name='loria')
    dataset = dataset.shuffle(seed=42)
    dataset = dataset.rename_column("Utterance", "utt")
    massive_fr = dataset.rename_column("Label", "intent")

    dataset_dict = {
    "French2": massive_fr,
    "English2": massive_en,
    "Farsi": massive_fa,
  }
    return dataset_dict


In [ ]:
def check_directory_status(
    dataset_dict,
    resume_from_checkpoint,
    path_to_checkpoint,
    Combination,
    Dataset_code,
    num_epochs,
    Saved_prop_task1: None,
    Saved_prop_task2: None,
    Mode = 1):

  '''
  Here we check the status of our directory in terms of having .csv files incorporating the probablities of the saved epochs.
  As we want to continue from the last saved epoch, we have to save the probabilities as well (starting from the last saved row in our numpy array)
  -------------------------------------------------------------------------------------------------------------------------------------------------
  Mode is a flag to save or load the array of probabilities. If Mode == 1, we are in the loading mode, otherwise we are in the saving mode.
  '''
  if isinstance(Dataset_code, list):

    if Combination == 0:
        # paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_French.csv')] + [os.path.join(path_to_checkpoint, 'Prob_per_epoch_English.csv')]
        paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_English2.csv')] + [os.path.join(path_to_checkpoint, 'Prob_per_epoch_French2.csv')] #??
    elif Combination == 1:
        paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_English2.csv')] + [os.path.join(path_to_checkpoint, 'Prob_per_epoch_Farsi.csv')]
    else:
        paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_French2.csv')] + [os.path.join(path_to_checkpoint, 'Prob_per_epoch_Farsi.csv')]


    if Mode:

        if os.path.exists(paths_to_check[0]) and resume_from_checkpoint is not None:  # Checking for one of  the paths reltaed to each multi lingual mode would be enough.
            Prob_per_epoch_first_task = loadtxt(paths_to_check[0], delimiter=',')
            Prob_per_epoch_second_task = loadtxt(paths_to_check[1], delimiter=',')
            if (Combination == 0):
                Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 151, 151)
                Prob_per_epoch_second_task = Prob_per_epoch_second_task.reshape(Prob_per_epoch_second_task.shape[0], Prob_per_epoch_second_task.shape[1] // 31, 31)
            elif (Combination == 1):
                Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 151, 151)
                Prob_per_epoch_second_task = Prob_per_epoch_second_task.reshape(Prob_per_epoch_second_task.shape[0], Prob_per_epoch_second_task.shape[1] // 60, 60)
            else:
                Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 31, 31)
                Prob_per_epoch_second_task = Prob_per_epoch_second_task.reshape(Prob_per_epoch_second_task.shape[0], Prob_per_epoch_second_task.shape[1] // 60, 60)

        else:
            if (Combination == 0):
                Prob_per_epoch_first_task = np.zeros((num_epochs,len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 151)) #151 labels
                Prob_per_epoch_second_task = np.zeros((num_epochs,len(dataset_dict[list(dataset_dict.keys())[1]]['train']), 31)) #31 labels
            elif (Combination == 1):
                Prob_per_epoch_first_task = np.zeros((num_epochs,len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 151)) #151 labels
                Prob_per_epoch_second_task = np.zeros((num_epochs,len(dataset_dict[list(dataset_dict.keys())[1]]['train']), 60)) #60 labels
            else:
                Prob_per_epoch_first_task = np.zeros((num_epochs,len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 31)) #31 labels
                Prob_per_epoch_second_task = np.zeros((num_epochs,len(dataset_dict[list(dataset_dict.keys())[1]]['train']), 60)) #60 labels

        return Prob_per_epoch_first_task, Prob_per_epoch_second_task

    else:

        savetxt(paths_to_check[0], Saved_prop_task1.reshape(Saved_prop_task1.shape[0], -1), delimiter=',')
        savetxt(paths_to_check[1], Saved_prop_task2.reshape(Saved_prop_task2.shape[0],-1), delimiter=',')
    # print("saving to: ", paths_to_check)
    # Prob_per_epoch_Paraphrase = np.zeros((NUM_EPOCHS,len(dataset_dict["mrpc"]['train']),OUTPUT_DIM)) #2 labels

  else:

    if Dataset_code == 0:
      paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_French2.csv')]
    elif Dataset_code == 1:
      paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_English2.csv')]
    else:
      paths_to_check = [os.path.join(path_to_checkpoint, 'Prob_per_epoch_Farsi.csv')]


  if Mode:

    if os.path.exists(paths_to_check[0]) and resume_from_checkpoint is not None:  # Checking for one of the paths associated with each multi task mode would be enough.

      if isinstance(Dataset_code, list):

        Prob_per_epoch_first_task = loadtxt(paths_to_check[0], delimiter=',')
        Prob_per_epoch_second_task = loadtxt(paths_to_check[1], delimiter=',')
        if (Combination == 0):
            Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 151, 151)
            Prob_per_epoch_second_task = Prob_per_epoch_second_task.reshape(Prob_per_epoch_second_task.shape[0], Prob_per_epoch_second_task.shape[1] //  31,  31)
        elif (Combination == 1):
            Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 151, 151)
            Prob_per_epoch_second_task = Prob_per_epoch_second_task.reshape(Prob_per_epoch_second_task.shape[0], Prob_per_epoch_second_task.shape[1] //  60,  60)
        else:
            Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 31, 31)
            Prob_per_epoch_second_task = Prob_per_epoch_second_task.reshape(Prob_per_epoch_second_task.shape[0], Prob_per_epoch_second_task.shape[1] //  60,  60)

      else:

        Prob_per_epoch_first_task = loadtxt(paths_to_check[0], delimiter=',')
        if (Dataset_code == 0):
            Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 31, 31)
        elif (Dataset_code == 1):
            Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 151, 151)
        else:
            Prob_per_epoch_first_task = Prob_per_epoch_first_task.reshape(Prob_per_epoch_first_task.shape[0], Prob_per_epoch_first_task.shape[1] // 60, 60)
        Prob_per_epoch_second_task =  None

    else:

      if isinstance(Dataset_code, list):
        if (Combination == 0):
            Prob_per_epoch_first_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 151)) #151 labels
            Prob_per_epoch_second_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[1]]['train']), 31)) #31 labels
        elif (Combination == 1):
            Prob_per_epoch_first_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 151)) #151 labels
            Prob_per_epoch_second_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[1]]['train']), 60)) #60 labels
        else:
            Prob_per_epoch_first_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 31)) #31 labels
            Prob_per_epoch_second_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[1]]['train']), 60)) #60 labels

      else:
        if (Dataset_code == 0):
            Prob_per_epoch_first_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 31)) #31 labels
        elif (Dataset_code == 1):
            Prob_per_epoch_first_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 151)) #151 labels
        else:
            Prob_per_epoch_first_task = np.zeros((num_epochs, len(dataset_dict[list(dataset_dict.keys())[0]]['train']), 60)) #60 labels
        Prob_per_epoch_second_task = None


    return Prob_per_epoch_first_task, Prob_per_epoch_second_task

  else:

    if isinstance(Dataset_code, list):
      savetxt(paths_to_check[0], Saved_prop_task1.reshape(Saved_prop_task1.shape[0], -1), delimiter=',')
      savetxt(paths_to_check[1], Saved_prop_task2.reshape(Saved_prop_task2.shape[0],-1), delimiter=',')
    else:
      savetxt(paths_to_check[0], Saved_prop_task1.reshape(Saved_prop_task1.shape[0], -1), delimiter=',')
    # Prob_per_epoch_Paraphrase = np.zeros((NUM_EPOCHS,len(dataset_dict["mrpc"]['train']),OUTPUT_DIM)) #2 labels

IndentationError: ignored

In [ ]:
def create_FRG_path(dataset_code, Fine_tune, Combination, dfs = None, Test = 0, Mode = 1, Actual_path = "/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)", FRG_str = "prob_per_10epochs"):
  '''
  dataset_code is dependant on the type of learning procedure (Single / Multi lingual); if we are in a single lingual mode, dataset_code is a single integer ranging from 0 to 2,
  otherwise, meaning we are in the multi-lingual mode, the dataset_code is a list consisting of two integers ranging from 0 to 2.
  -------------------------------------------------------------------------------------------------------------------------------
  Mode is set to switch between saving or loading mode, if Mode == 1, we are in the loading mode, otherwise, we are in the saving mode.
  '''
  task_name = ['French2_', "English2_", "Farsi_"]
  post_MTL_str = ["_FREN2", "_ENFA2", "_FRFA2"]
  prob = [None, None]
  if isinstance(dataset_code, list): # we are in the multi-lingual mode.
    Selected_path = (os.path.join(Actual_path, "Multi Lingual/" + ("Fine-tune/" if Fine_tune else "") + task_name[dataset_code[0]] + "MLL_" + (FRG_str if Mode else "FRG_Info") + post_MTL_str[Combination] + ("_Test" if Test else "") + (".pkl" if Mode else ".csv")),
                    os.path.join(Actual_path, "Multi Lingual/" + ("Fine-tune/" if Fine_tune else "") + task_name[dataset_code[1]] + "MLL_" + (FRG_str if Mode else "FRG_Info") + post_MTL_str[Combination] + ("_Test" if Test else "") + (".pkl" if Mode else ".csv")))

    if Mode:
      with open(Selected_path[0],'rb') as f1: prob[0] = pickle.load(f1)
      with open(Selected_path[1],'rb') as f2: prob[1] = pickle.load(f2)
    else:
      dfs[0].to_csv(Selected_path[0])
      dfs[1].to_csv(Selected_path[1])

  else: # we are in the single-lingual mode.
    Selected_path = os.path.join(Actual_path, "Single Lingual/" + ("Fine-tune/" if Fine_tune else "") + task_name[dataset_code] + (FRG_str if Mode else "FRG_Info") + (".pkl" if Mode else ".csv"))
    if Mode:
      with open(Selected_path,'rb') as f: prob = pickle.load(f)
    else:
      dfs.to_csv(Selected_path)

  return prob


In [ ]:
def save_final_prop(Dataset_code, Fine_tune, Combination, prob_first_task, prob_second_task):

  Actual_path = "/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual"
  Fine_tune_path = "/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/Fine-tune"

  if isinstance(Dataset_code, list):

    if Fine_tune:
        Selected_path = Fine_tune_path
    else:
        Selected_path = Actual_path

    if Combination == 0:

        # first_path = os.path.join(Selected_path, "French_" + "MLL_prob_per_10epochs_" + "FREN" + ".pkl")
        # second_path = os.path.join(Selected_path, "English_" + "MLL_prob_per_10epochs_" + "FREN" + ".pkl")

        first_path = os.path.join(Selected_path, "English2_" + "MLL_prob_per_10epochs_" + "FREN2" + ".pkl")
        second_path = os.path.join(Selected_path, "French2_" + "MLL_prob_per_10epochs_" + "FREN2" + ".pkl")

    elif Combination == 1:

        first_path = os.path.join(Selected_path, "English2_" + "MLL_prob_per_10epochs_" + "ENFA2" + ".pkl")
        second_path = os.path.join(Selected_path, "Farsi_" + "MLL_prob_per_10epochs_" + "ENFA2" + ".pkl")

    else:

        first_path = os.path.join(Selected_path, "French2_" + "MLL_prob_per_10epochs_" + "FRFA2" + ".pkl")
        second_path = os.path.join(Selected_path, "Farsi_" + "MLL_prob_per_10epochs_" + "FRFA2" + ".pkl")


    with open(first_path, 'wb') as f:

        pickle.dump(prob_first_task, f)

    with open(second_path,'wb') as f:

        pickle.dump(prob_second_task, f)

  else:
    Selected_path =  "/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/" + "Single Lingual"

    if Dataset_code == 0:
      first_path = os.path.join(Selected_path, "French2_" + "prob_per_10epochs" + ".pkl")
    elif Dataset_code == 1:
      first_path = os.path.join(Selected_path, "English2_" + "prob_per_10epochs" + ".pkl")
    else:
      first_path = os.path.join(Selected_path, "Farsi_" + "prob_per_10epochs" + ".pkl")

    with open(first_path, 'wb') as f:

      pickle.dump(prob_first_task, f)

In [ ]:
def create_batch(multitask_model, features_dict, trainer, tokenizer, max_length, val_len, batch_size, task_name):
  acc = 0
  multitask_model.taskmodels_dict[task_name].eval()
  with torch.no_grad():
    for index in range(0, val_len, batch_size):
        batch = features_dict[task_name]["test"]["utt"][index : min(index + batch_size, val_len)]

        labels = features_dict[task_name]["test"]["intent"][index : min(index + batch_size, val_len)]
        inputs = tokenizer.batch_encode_plus(batch, max_length=max_length, pad_to_max_length=True, return_tensors='pt').to(trainer.args.device)
        logits = multitask_model(task_name, **inputs)[0]
        predictions = torch.argmax(torch.FloatTensor(torch.softmax(logits, dim=1).detach().cpu().tolist()), dim=1)
        acc += sum(np.array(predictions) == np.array(labels))

  return acc

In [ ]:
def multitask_eval_fn(multitask_model, features_dict, model_name, dataset_dict, trainer, tokenizer, max_length, batch_size, eval_dataset = 'test'):
    preds_dict = {}
    for task_name in list(dataset_dict.keys()):
        val_len = len(features_dict[task_name][eval_dataset])
        acc_per_task = create_batch(multitask_model,
                                    features_dict,
                                    trainer,
                                    tokenizer,
                                    max_length,
                                    val_len,
                                    batch_size,
                                    task_name)

        acc_per_task= acc_per_task / val_len
        print("Language: %s \t Accuracy: %.4f" % (task_name, acc_per_task))

In [ ]:
def extract_forgettable_samples(prob_arr, dataset, dataset_code, Fine_tune):

  true_label = dataset['train']['intent']


  forgettable_info = {'sample_index':[],'forgetting_events':[],'Match_label_indexs':[]}
  for i in range(prob_arr.shape[1]):
    forgetting_events = 0
    prob_per_sample = prob_arr[:,i,:]
    label_per_sample = np.argmax(prob_per_sample,axis=1)
    Match_label_indexs = np.where(label_per_sample == true_label[i])[0]

    if Match_label_indexs.shape[0] == 0: # no correct prediction
      forgettable_info['sample_index'].append((i if Fine_tune == 0 or isinstance(dataset_code, int) else dataset['train']['index'][i]))
      forgettable_info['forgetting_events'].append(0)
      forgettable_info['Match_label_indexs'].append(Match_label_indexs)

    else:
      for item in (Match_label_indexs + 1):
        if item not in Match_label_indexs and item < prob_arr.shape[0]:
            forgetting_events += 1
      if forgetting_events > 0:
        forgettable_info['sample_index'].append((i if Fine_tune == 0 or isinstance(dataset_code, int) else dataset['train']['index'][i]))
        forgettable_info['forgetting_events'].append(forgetting_events)
        forgettable_info['Match_label_indexs'].append(Match_label_indexs)
      else:
        continue

  return forgettable_info


In [ ]:
def return_dataset(Dataset_code, Combination, Fine_tune = 1, drop_unlearned = 0):

  '''
  |----------------------------------------------------------------------------------------------------------------------|
  |dataset_code is an indication of each of the available tasks (datasets).                                              |
  |dataset_code = 0: the corresponding French2 task/dataset,                                                           |
  |dataset_code = 1: the corresponding English2 task/dataset,                                                          |
  |dataset_code = 2: the corresponding Farsi task/dataset.                                                            |
  |----------------------------------------------------------------------------------------------------------------------|
  |Combination is an integer number ranging from 0 to 2, pointing out to one of the possible options in multi-lingual mode. |
  |Combination = 0: We are considering => English2 & French2,                                                              |
  |Combination = 1: We are considering => English2 & Farsi,                                                               |
  |Combination = 2: We are considering => French2 & Farsi.                                                                |
  |----------------------------------------------------------------------------------------------------------------------|
  |If Fine-tune = 1 => Load the forgettable examples only dataset,                                                       |
  |If Fine-tune = 0 => Load the initial dataset                                                                          |
  |----------------------------------------------------------------------------------------------------------------------|
  '''
  if isinstance(Dataset_code, list):

    if Fine_tune:
        dataset_dict = load_dict()
        # Checking Modes
        if Combination == 0:
            df_multi_E_EP = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/English2_MLL_FRG_Info_FREN2.csv")
            df_multi_P_EP = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/French2_MLL_FRG_Info_FREN2.csv")

            # droping unlearned samples
            if (drop_unlearned):
                df_multi_E_EP = df_multi_E_EP[df_multi_E_EP['Match_label_indexs'] != '[]']
                df_multi_P_EP = df_multi_P_EP[df_multi_P_EP['Match_label_indexs'] != '[]']

            # Configs for English2
            df_train_Entailment = pd.DataFrame(dataset_dict["English2"]['train'])
            #   df_train_Entailment['label'] = (df_train_Entailment['label'] == 'entails').astype(int)
            df_train_Entailment['index'] = pd.Series(range(len(df_train_Entailment)))
            df_train_Entailment = df_train_Entailment[(pd.concat([df_train_Entailment['index'], df_multi_E_EP['sample_index']]).value_counts() == 2)]
            df_test_Entailment = pd.DataFrame(dataset_dict["English2"]['test'])
            #   df_test_Entailment['label'] = (df_test_Entailment['label'] == 'entails').astype(int)
            df_validation_Entailment =  pd.DataFrame(dataset_dict["English2"]['validation'])
            #   df_validation_Entailment ['label'] = (df_validation_Entailment ['label'] == 'entails').astype(int)

            dataset_dict["English2"]['train'] = Dataset.from_pandas(df_train_Entailment, preserve_index=False)
            dataset_dict["English2"]['test'] = Dataset.from_pandas(df_test_Entailment, preserve_index=False)
            dataset_dict["English2"]['validation'] = Dataset.from_pandas(df_validation_Entailment, preserve_index=False)
            # Configs for French2
            df_train_Paraphrase = pd.DataFrame(dataset_dict["French2"]['train'])
            df_train_Paraphrase['index'] = pd.Series(range(len(df_train_Paraphrase)))
            df_train_Paraphrase = df_train_Paraphrase[(pd.concat([df_train_Paraphrase['index'],df_multi_P_EP['sample_index']]).value_counts() == 2)]
            df_test_Paraphrase = pd.DataFrame(dataset_dict["French2"]['test'])
            df_validation_Paraphrase = pd.DataFrame(dataset_dict["French2"]['validation'])

            dataset_dict["French2"]['train'] = Dataset.from_pandas(df_train_Paraphrase, preserve_index=False)
            dataset_dict["French2"]['test'] = Dataset.from_pandas(df_test_Paraphrase, preserve_index=False)
            dataset_dict["French2"]['validation'] = Dataset.from_pandas(df_validation_Paraphrase, preserve_index=False)
            # return the dataset_dict for the specified combination
            my_dict = {}
            my_dict["English2"] = dataset_dict["English2"]
            my_dict["French2"] = dataset_dict["French2"]

            return my_dict


        elif Combination == 1:
            #   df_multi_S_ES = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(csv files)/Multi Lingual/Farsi_Forgettables_Info_ENFA.csv")
            df_multi_S_ES = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/Farsi_MLL_FRG_Info_ENFA2.csv")
            #   df_multi_E_ES = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(csv files)/Multi Lingual/English_Forgettables_Info_ENFA.csv")
            df_multi_E_ES = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/English2_MLL_FRG_Info_ENFA2.csv")

            # droping unlearned samples
            if (drop_unlearned):
                df_multi_S_ES = df_multi_S_ES[df_multi_S_ES['Match_label_indexs'] != '[]']
                df_multi_E_ES = df_multi_E_ES[df_multi_E_ES['Match_label_indexs'] != '[]']

            # Configs for English2
            df_train_Entailment = pd.DataFrame(dataset_dict["English2"]['train'])
            #   df_train_Entailment['label'] = (df_train_Entailment['label'] == 'entails').astype(int)
            df_train_Entailment['index'] = pd.Series(range(len(df_train_Entailment)))
            df_train_Entailment = df_train_Entailment[(pd.concat([df_train_Entailment['index'],df_multi_E_ES['sample_index']]).value_counts() == 2)]
            df_test_Entailment = pd.DataFrame(dataset_dict["English2"]['test'])
            #   df_test_Entailment['label'] = (df_test_Entailment['label'] == 'entails').astype(int)
            df_validation_Entailment =  pd.DataFrame(dataset_dict["English2"]['validation'])
            #   df_validation_Entailment ['label'] = (df_validation_Entailment ['label'] == 'entails').astype(int)

            dataset_dict["English2"]['train'] = Dataset.from_pandas(df_train_Entailment, preserve_index=False)
            dataset_dict["English2"]['test'] = Dataset.from_pandas(df_test_Entailment, preserve_index=False)
            dataset_dict["English2"]['validation'] = Dataset.from_pandas(df_validation_Entailment, preserve_index=False)
            # Configs for Farsi
            df_train_Sentiment = pd.DataFrame(dataset_dict["Farsi"]['train'])
            df_train_Sentiment['index'] = pd.Series(range(len(df_train_Sentiment)))
            df_train_Sentiment = df_train_Sentiment[(pd.concat([df_train_Sentiment['index'],df_multi_S_ES['sample_index']]).value_counts() == 2)]
            df_test_Sentiment = pd.DataFrame(dataset_dict["Farsi"]['test'])
            df_validation_Sentiment =  pd.DataFrame(dataset_dict["Farsi"]['validation'])

            dataset_dict["Farsi"]['train'] = Dataset.from_pandas(df_train_Sentiment, preserve_index=False)
            dataset_dict["Farsi"]['test'] = Dataset.from_pandas(df_test_Sentiment, preserve_index=False)
            dataset_dict["Farsi"]['validation'] = Dataset.from_pandas(df_validation_Sentiment, preserve_index=False)
            # return the dataset_dict for the specified combination
            my_dict = {}
            my_dict["English2"] = dataset_dict["English2"]
            my_dict["Farsi"] = dataset_dict["Farsi"]

            return my_dict


        elif Combination == 2:
            #   df_multi_P_PS = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(csv files)/Multi Lingual/French_Forgettables_Info_FRFA.csv")
            df_multi_P_PS = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/French2_MLL_FRG_Info_FRFA2.csv")
            #   df_multi_S_PS = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(csv files)/Multi Lingual/Farsi_Forgettables_Info_FRFA.csv")
            df_multi_S_PS = pd.read_csv("/content/drive/MyDrive/NLP Bachelors' Project/FRG_Info(pkl files)/Multi Lingual/Farsi_MLL_FRG_Info_FRFA2.csv")

            # droping unlearned samples
            if (drop_unlearned):
                df_multi_P_PS = df_multi_P_PS[df_multi_P_PS['Match_label_indexs'] != '[]']
                df_multi_S_PS = df_multi_S_PS[df_multi_S_PS['Match_label_indexs'] != '[]']

            # Configs for French2
            df_train_Paraphrase = pd.DataFrame(dataset_dict["French2"]['train'])
            df_train_Paraphrase['index'] = pd.Series(range(len(df_train_Paraphrase)))
            df_train_Paraphrase = df_train_Paraphrase[(pd.concat([df_train_Paraphrase['index'],df_multi_P_PS['sample_index']]).value_counts() == 2)]
            df_test_Paraphrase = pd.DataFrame(dataset_dict["French2"]['test'])
            df_validation_Paraphrase =  pd.DataFrame(dataset_dict["French2"]['validation'])

            dataset_dict["French2"]['train'] = Dataset.from_pandas(df_train_Paraphrase, preserve_index=False)
            dataset_dict["French2"]['test'] = Dataset.from_pandas(df_test_Paraphrase, preserve_index=False)
            dataset_dict["French2"]['validation'] = Dataset.from_pandas(df_validation_Paraphrase, preserve_index=False)
            # Configs for Farsi
            df_train_Sentiment = pd.DataFrame(dataset_dict["Farsi"]['train'])
            df_train_Sentiment['index'] = pd.Series(range(len(df_train_Sentiment)))
            df_train_Sentiment = df_train_Sentiment[(pd.concat([df_train_Sentiment['index'],df_multi_S_PS['sample_index']]).value_counts() == 2)]
            df_test_Sentiment = pd.DataFrame(dataset_dict["Farsi"]['test'])
            df_validation_Sentiment =  pd.DataFrame(dataset_dict["Farsi"]['validation'])

            dataset_dict["Farsi"]['train'] = Dataset.from_pandas(df_train_Sentiment, preserve_index=False)
            dataset_dict["Farsi"]['test'] = Dataset.from_pandas(df_test_Sentiment, preserve_index=False)
            dataset_dict["Farsi"]['validation'] = Dataset.from_pandas(df_validation_Sentiment, preserve_index=False)
            # return the dataset_dict for the specified combination
            my_dict = {}
            my_dict["French2"] = dataset_dict["French2"]
            my_dict["Farsi"] = dataset_dict["Farsi"]

            return my_dict

    else:
        dataset_dict = load_dict()
        # Checking modes
        if Combination == 0:
            return {"English2": dataset_dict['English2'],"French2": dataset_dict["French2"]}
        elif Combination == 1:
            return {"English2": dataset_dict['English2'], "Farsi": dataset_dict["Farsi"]}
        elif Combination == 2:
            return {"French2": dataset_dict["French2"], "Farsi": dataset_dict["Farsi"]}

  else:
    dataset_dict = load_dict()

    if Dataset_code == 0:
      return {"French2": dataset_dict["French2"]}

    elif Dataset_code == 1:
      return {"English2": dataset_dict["English2"]}

    else:
      return {"Farsi": dataset_dict["Farsi"]}
